# API KEY

In [11]:
# API Key Deepseek Chimera
# --> https://openrouter.ai/settings/keys
OPENROUTER_API_KEY = "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"  # <--  API KEY

In [12]:
PEXELS_API_KEY = "mAeOeBxoIDpJba5LKFHLBNfp4ektHkYJYLAQhemwTJEBX9rmjTsTJmo8"

# Library

In [13]:
import json
import os
import faiss
import numpy as np
import requests
from sentence_transformers import SentenceTransformer
from gtts import gTTS

In [14]:
# Load FAISS Index
index = faiss.read_index("output/faiss_index.bin")

# Load Mapping sumber chunk
with open("output/chunk_mapping.json", "r", encoding="utf-8") as f:
    chunk_sources = json.load(f)

# Load ulang semua chunk text
with open("output/cleaned_text/epub_chunks_translated.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Gabungkan semua chunk
all_chunks = []
for book_title, chunks in data.items():
    for chunk in chunks:
        if chunk.strip():
            all_chunks.append(chunk)

# Load model embedding
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


In [15]:
def search_best_chunk(question, top_k=1):
    """Cari chunk paling relevan untuk pertanyaan."""
    question_embedding = model.encode([question]).astype('float32')
    D, I = index.search(question_embedding, top_k)

    results = []
    for idx in I[0]:
        if idx < len(all_chunks):
            results.append((chunk_sources[idx], all_chunks[idx]))
    return results


In [39]:
import requests
import re

def summarize_with_deepseek(text, api_key=OPENROUTER_API_KEY, output_language="English"):
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    prompt = (
        f"Write a compelling final narration in {output_language} for a 1-minute YouTube video, "
        f"based on the following text. Use warm, natural, inspiring language. "
        f"Do NOT include your internal thoughts or step-by-step process. Just give the final script:\n\n{text}"
    )

    data = {
        "model": "tngtech/deepseek-r1t-chimera:free",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.5
    }

    try:
        response = requests.post(url, headers=headers, json=data)
        if response.status_code != 200:
            print(f"❌ ERROR: {response.status_code} - {response.text}")
            return "[SUMMARY FAILED]"

        content = response.json()['choices'][0]['message']['content']

        # error catch
        if "Narration:" in content:
            content = content.split("Narration:")[-1].strip()
        else:
            match = re.search(r"\*\*(.*?)\*\*", content, re.DOTALL)
            if match:
                content = match.group(1).strip()

        # Bersihkan
        if "Let me" in content or "First," in content[:80]:
            # Potong kalimat pertama yang berisi kutipan atau awalan puitis
            sentences = content.split('\n')
            for i, line in enumerate(sentences):
                if line.strip().startswith("\"") or line.strip().startswith("In the"):
                    content = "\n".join(sentences[i:])
                    break

        return content.strip()

    except Exception as e:
        print(f"❌ ERROR: {e}")
        return "[SUMMARY FAILED]"


In [40]:
def text_to_speech(text, output_audio_path="output/answer_audio.mp3"):
    tts = gTTS(text=text, lang='en')
    tts.save(output_audio_path)
    return output_audio_path


In [42]:
# Pertanyaan user
# question = "What does Islam say about the importance of youth?"
question = "What does Islam say importan pray?"

# Step 1: Cari Chunk
results = search_best_chunk(question, top_k=1)
source, answer = results[0]

print(f"📖 Source: {source}")
print(f"📝 Original Answer: {answer}")

# Step 2: Summarize dengan DeepSeek
summarized_answer = summarize_with_deepseek(answer)
print(f"📝 Summarized Answer: {summarized_answer}")

# Step 3: Buat Audio
audio_path = text_to_speech(summarized_answer)


📖 Source: تيسير اللطيف المنان في خلاصة تفسير القرآن - ط الأوقاف السعودية - Chunk 34
📝 Original Answer: And do not complete the wet from it, you will spend, and you will take it, except that you will be overwhelmed by it and know that God is the rich of Hamid: [267] His harvest [Al -Anam: 141] God has gathered in his book in many verses between the matter to establish prayer and pay zakat; Because they are two common that they are one of the most important assumptions of religion, and the great buildings of Islam, and faith is only done with them, and whoever prayed and zakat was a residing in his religion, and whoever lost them was because of the other of his religion was lost, so the prayer in it is complete devotion to the idol, which is the balance of faith, and zakat in it is charity to the creatures, which is page: 75 unknown to the proof of faith Zakat, and Abu Bakr, may God be pleased with him, said: “For a difference between a difference between prayer and zakat.” The other ver

In [19]:
# import os
# from moviepy.editor import AudioFileClip, TextClip, CompositeVideoClip, ColorClip, concatenate_videoclips
# from moviepy.config import change_settings
# import textwrap

# change_settings({"IMAGEMAGICK_BINARY": r"C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe"})


# def split_text_to_chunks(text, max_chars=120):
#     """
#     Memecah teks menjadi potongan-potongan pendek agar cocok untuk satu layar.
#     """
#     return textwrap.wrap(text, width=max_chars)


# def create_video_with_audio_and_text(audio_path, text, output_path="output/final_video.mp4"):
#     # Load audio
#     audio = AudioFileClip(audio_path)
#     duration = audio.duration

#     # Pecah teks jadi beberapa bagian
#     chunks = split_text_to_chunks(text)
#     num_chunks = len(chunks)
#     chunk_duration = duration / num_chunks  # Durasi tiap segmen

#     video_clips = []

#     for i, chunk in enumerate(chunks):
#         background = ColorClip(size=(1280, 720), color=(0, 0, 0), duration=chunk_duration)

#         text_clip = TextClip(
#             txt=chunk,
#             fontsize=48,
#             color='white',
#             font='Arial-Bold',
#             size=(1100, None),
#             method='caption'
#         ).set_duration(chunk_duration).set_position(("center", "bottom"))

#         video_clip = CompositeVideoClip([background, text_clip])
#         video_clips.append(video_clip)

#     # Gabungkan semua klip
#     final_video = concatenate_videoclips(video_clips).set_audio(audio)

#     os.makedirs(os.path.dirname(output_path), exist_ok=True)
#     final_video.write_videofile(output_path, fps=24)

# # Contoh pemanggilan (pastikan audio_path dan summarized_answer didefinisikan)
# create_video_with_audio_and_text(audio_path, summarized_answer)
# print("✅ Video berhasil dibuat.")

In [35]:
import os
import requests
from io import BytesIO
from PIL import Image
from moviepy.editor import AudioFileClip, TextClip, CompositeVideoClip, ImageClip, concatenate_videoclips
from moviepy.config import change_settings
import textwrap
import hashlib

change_settings({"IMAGEMAGICK_BINARY": r"C:\\Program Files\\ImageMagick-7.1.1-Q16-HDRI\\magick.exe"})

def split_text_to_chunks(text, max_chars=120):
    return textwrap.wrap(text, width=max_chars)

def sanitize_filename(text):
    return hashlib.md5(text.encode('utf-8')).hexdigest()

def download_single_image_from_pexels(query, index=0):
    headers = {
        "Authorization": PEXELS_API_KEY
    }
    response = requests.get(f"https://api.pexels.com/v1/search?query={query}&per_page=1", headers=headers)
    data = response.json()
    
    if "photos" in data and data["photos"]:
        image_url = data["photos"][0]["src"]["landscape"]
        img_data = requests.get(image_url).content
        image = Image.open(BytesIO(img_data)).resize((1280, 720))
        image_path = f"temp_bg_{sanitize_filename(query)}_{index}.jpg"
        image.save(image_path)
        return image_path
    else:
        # fallback background
        black_img_path = f"temp_bg_black_{index}.jpg"
        Image.new("RGB", (1280, 720), (0, 0, 0)).save(black_img_path)
        return black_img_path

def create_video_with_audio_and_text(audio_path, text, output_path="output/final_video.mp4"):
    audio = AudioFileClip(audio_path)
    duration = audio.duration

    chunks = split_text_to_chunks(text)
    num_chunks = len(chunks)
    chunk_duration = duration / num_chunks

    video_clips = []
    temp_image_paths = []

    for i, chunk in enumerate(chunks):
        query = chunk.split(".")[0]
        bg_path = download_single_image_from_pexels(query, index=i)
        temp_image_paths.append(bg_path)  # simpan path untuk dihapus nanti

        background = ImageClip(bg_path).set_duration(chunk_duration)

        text_clip = TextClip(
            txt=chunk,
            fontsize=48,
            color='white',
            font='Arial-Bold',
            size=(1100, None),
            method='caption'
        ).set_duration(chunk_duration).set_position(("center", "bottom"))

        video_clip = CompositeVideoClip([background, text_clip])
        video_clips.append(video_clip)

    final_video = concatenate_videoclips(video_clips).set_audio(audio)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    final_video.write_videofile(output_path, fps=24)

    # Hapus semua gambar setelah selesai
    for path in temp_image_paths:
        if os.path.exists(path):
            os.remove(path)

# Contoh pemanggilan
create_video_with_audio_and_text(audio_path, summarized_answer)
print("✅ Video berhasil dibuat & gambar sementara telah dihapus.")


Moviepy - Building video output/final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output/final_video.mp4



Moviepy - Done !
Moviepy - video ready output/final_video.mp4
✅ Video berhasil dibuat & gambar sementara telah dihapus.
